In [1]:
import os
import numpy as np
import natsort
from PIL import Image

file_path = './heatmap_case2_0.2'
file_list = os.listdir(file_path)
file_list = natsort.natsorted(file_list, key=None, reverse=False, alg=0)

num_of_files = len(file_list)

# Assuming all images have the same height and width
first_image = Image.open(os.path.join(file_path, file_list[0]))
height, width = np.array(first_image).shape

# Create an empty NumPy array to store the images
image_array = np.empty((num_of_files, height, width))

# Iterate through the file list and load images
for i, file_name in enumerate(file_list):
    image = Image.open(os.path.join(file_path, file_name))
    image_array[i] = np.array(image)

# image_array now contains the images as NumPy arrays
print(image_array.shape)  # (num_of_files, height, width)
print(file_list[:5])

(1300, 9, 64)
['train0.png', 'train1.png', 'train2.png', 'train3.png', 'train4.png']


In [2]:
import os
import numpy as np
from PIL import Image


num_of_frames = 13  # Specify the desired number of frames

# Assuming all images have the same height and width
first_image = Image.open(os.path.join(file_path, file_list[0]))
height, width = np.array(first_image).shape[:2]

# Create an empty NumPy array to store the images
dataset = np.empty((num_of_files-num_of_frames+1, num_of_frames, height, width))

# Iterate through the file list and load images
for i in range(num_of_files-num_of_frames+1):
    frame_images = file_list[i : i+num_of_frames]
    for j, file_name in enumerate(frame_images):
        if i < 2:
            print(file_name)
        image = Image.open(os.path.join(file_path, file_name))
        dataset[i, j] = np.array(image)
    if i < 2:
        print()

# dataset now contains the images bound into frames
print(dataset.shape)  # (num_of_files // num_of_frames, num_of_frames, height, width)

train0.png
train1.png
train2.png
train3.png
train4.png
train5.png
train6.png
train7.png
train8.png
train9.png
train10.png
train11.png
train12.png

train1.png
train2.png
train3.png
train4.png
train5.png
train6.png
train7.png
train8.png
train9.png
train10.png
train11.png
train12.png
train13.png

(1288, 13, 9, 64)


In [3]:
# Add a channel dimension since the images are grayscale.
dataset = np.expand_dims(dataset, axis=-1)
dataset.shape #(num_of_samples, frames, height, width, channels)

(1288, 13, 9, 64, 1)

In [4]:
import numpy as np

np.random.seed(42)

# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset.shape[0])]
val_index = indexes[int(0.9 * dataset.shape[0]) :]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]

# Normalize the data to the 0-1 range.
train_dataset = train_dataset / 255
val_dataset = val_dataset / 255

print(train_dataset.shape, val_dataset.shape)

(1159, 13, 9, 64, 1) (129, 13, 9, 64, 1)


# RNN
1. 각 MEC 당 128 input -> 128개 output.
2. 과거 12개 관찰.

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers
from tensorflow.keras import models

import numpy as np

tf.config.run_functions_eagerly(True)

In [6]:
import numpy as np

window_size  = 12

def create_shifted_frames(data, mec_index): # data = (frames, mecs, movieId, gray_scale)
    tmp_data = data[:,mec_index,:,:]
    tmp_data = np.squeeze(tmp_data)
    tmp_data = tmp_data.transpose()

    return tmp_data

In [7]:
def get_model():
    inputs = layers.Input(shape=(window_size,1))
    encoder = layers.LSTM(100, activation="relu", return_sequences=False)(inputs)
    repeat = layers.RepeatVector(1)(encoder)
    decoder = layers.LSTM(100, activation='relu', return_sequences=True)(repeat)
    outputs = layers.TimeDistributed(layers.Dense(1))(decoder)

    model = models.Model(inputs, outputs)

    model.compile(optimizer='adam', loss='mse')

    return model

tf.keras.backend.clear_session()
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 100)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 1, 100)            80400     
                                                                 
 time_distributed (TimeDistr  (None, 1, 1)             101       
 ibuted)                                                         
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable

In [8]:
from IPython.display import clear_output

# observe_mec_index = 0

In [9]:
X_train = [[] for _ in range(train_dataset.shape[2])]
Y_train = [[] for _ in range(train_dataset.shape[2])]
for day_index, oneday_dataset in enumerate(train_dataset):
    for mec_index in range(oneday_dataset.shape[1]):
        clear_output(wait=True)
        print(day_index, ' train ...')
        
        try:
            if mec_index != observe_mec_index:
                continue
        except:
            print('no observe mec index')

        print(mec_index,' 번째 mec 학습중 ...')
        frame_dataset = create_shifted_frames(oneday_dataset, mec_index)
        print('frame shape : ', frame_dataset.shape)

        X,Y = np.empty((1,12,1)), np.empty((1))
        for movieId_index, movieId in enumerate(frame_dataset):
            print(movieId_index, ' movie  train ... ', movieId.shape)
            x = movieId[:-1]
            x = np.array([[x[i] for i in range(j,j+window_size)] for j in range(len(x)-window_size+1)])
            y = movieId[window_size:]
            x = np.expand_dims(x, axis=-1)
            #print('final X,Y shape : ',  x.shape, y.shape)
            X = np.append(X,x, axis=0)
            Y = np.append(Y,y, axis=0)

        X = np.delete(X, 0, axis=0)
        Y = np.delete(Y, 0, axis=0)
        try:
            X_train[mec_index].extend(X)
            Y_train[mec_index].extend(Y)
        except:
            X_train[mec_index] = np.append(X_train[mec_index], X, axis=0)
            Y_train[mec_index] = np.append(Y_train[mec_index], Y, axis=0)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
print(X_train.shape, Y_train.shape)

1158  train ...
no observe mec index
8  번째 mec 학습중 ...
frame shape :  (64, 13)
0  movie  train ...  (13,)
1  movie  train ...  (13,)
2  movie  train ...  (13,)
3  movie  train ...  (13,)
4  movie  train ...  (13,)
5  movie  train ...  (13,)
6  movie  train ...  (13,)
7  movie  train ...  (13,)
8  movie  train ...  (13,)
9  movie  train ...  (13,)
10  movie  train ...  (13,)
11  movie  train ...  (13,)
12  movie  train ...  (13,)
13  movie  train ...  (13,)
14  movie  train ...  (13,)
15  movie  train ...  (13,)
16  movie  train ...  (13,)
17  movie  train ...  (13,)
18  movie  train ...  (13,)
19  movie  train ...  (13,)
20  movie  train ...  (13,)
21  movie  train ...  (13,)
22  movie  train ...  (13,)
23  movie  train ...  (13,)
24  movie  train ...  (13,)
25  movie  train ...  (13,)
26  movie  train ...  (13,)
27  movie  train ...  (13,)
28  movie  train ...  (13,)
29  movie  train ...  (13,)
30  movie  train ...  (13,)
31  movie  train ...  (13,)
32  movie  train ...  (13,)
33  mov

In [10]:
with tf.device("/gpu:1"):
    for mec_index in range(train_dataset.shape[2]): # mecs만큼 반복
        print(mec_index,'mec 학습중 ...')
        rnn_model = get_model()
        print(rnn_model.summary())
        print(X_train[mec_index].shape, Y_train[mec_index].shape)
        rnn_model.fit(X_train[mec_index], Y_train[mec_index], epochs=20, validation_split=0.2, verbose=1, batch_size = 128)
        rnn_model.save('./rnn_models/MEC'+str(mec_index)+'rnn_'+file_path[-9:])

0 mec 학습중 ...
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 1)]           0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 100)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 1, 100)            80400     
                                                                 
 time_distributed_1 (TimeDis  (None, 1, 1)             101       
 tributed)                                                       
                                                                 
Total params: 121,301
Trainable params: 121,3

C:\Users\kwangwoon\anaconda3\envs\rnn-gpu\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


464/464 [==============================] - 50s 105ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 2/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 3/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 4/20
464/464 [==============================] - 50s 108ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 5/20
464/464 [==============================] - 51s 109ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 6/20
464/464 [==============================] - 50s 108ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 7/20
464/464 [==============================] - 51s 109ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 8/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 9/20
464/464 [==============================] - 50s 108ms/step - loss: 0.0245 - val_loss: 0.0251
Epoch 10/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0245

C:\Users\kwangwoon\anaconda3\envs\rnn-gpu\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


464/464 [==============================] - 51s 109ms/step - loss: 0.0237 - val_loss: 0.0234
Epoch 2/20
464/464 [==============================] - 51s 110ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 3/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 4/20
464/464 [==============================] - 49s 105ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 5/20
464/464 [==============================] - 49s 105ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 6/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0237 - val_loss: 0.0234
Epoch 7/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 8/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 9/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0236 - val_loss: 0.0234
Epoch 10/20
464/464 [==============================] - 48s 104ms/step - loss: 0.0236

...

464/464 [==============================] - 47s 101ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 2/20
464/464 [==============================] - 47s 100ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 3/20
464/464 [==============================] - 48s 103ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 4/20
464/464 [==============================] - 48s 103ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 5/20
464/464 [==============================] - 46s 100ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 6/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 7/20
464/464 [==============================] - 46s 100ms/step - loss: 0.0253 - val_loss: 0.0250
Epoch 8/20
464/464 [==============================] - 49s 106ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 9/20
464/464 [==============================] - 46s 98ms/step - loss: 0.0253 - val_loss: 0.0251
Epoch 10/20
464/464 [==============================] - 50s 107ms/step - loss: 0.0253 

C:\Users\kwangwoon\anaconda3\envs\rnn-gpu\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


464/464 [==============================] - 45s 97ms/step - loss: 0.0237 - val_loss: 0.0236
Epoch 2/20
464/464 [==============================] - 44s 95ms/step - loss: 0.0236 - val_loss: 0.0236
Epoch 3/20
464/464 [==============================] - 46s 99ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 4/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0236
Epoch 5/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 6/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 7/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 8/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 9/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 10/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0236 - val_los

C:\Users\kwangwoon\anaconda3\envs\rnn-gpu\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


464/464 [==============================] - 45s 96ms/step - loss: 0.0256 - val_loss: 0.0246
Epoch 2/20
464/464 [==============================] - 45s 96ms/step - loss: 0.0256 - val_loss: 0.0247
Epoch 3/20
464/464 [==============================] - 46s 99ms/step - loss: 0.0256 - val_loss: 0.0246
Epoch 4/20
464/464 [==============================] - 44s 96ms/step - loss: 0.0256 - val_loss: 0.0245
Epoch 5/20
464/464 [==============================] - 45s 98ms/step - loss: 0.0256 - val_loss: 0.0246
Epoch 6/20
464/464 [==============================] - 44s 95ms/step - loss: 0.0256 - val_loss: 0.0245
Epoch 7/20
464/464 [==============================] - 45s 96ms/step - loss: 0.0256 - val_loss: 0.0245
Epoch 8/20
464/464 [==============================] - 46s 98ms/step - loss: 0.0256 - val_loss: 0.0245
Epoch 9/20
464/464 [==============================] - 44s 96ms/step - loss: 0.0256 - val_loss: 0.0245
Epoch 10/20
464/464 [==============================] - 45s 97ms/step - loss: 0.0256 - val_los